In [0]:
 
 %pip install databricks-genai-inference
 %pip install --upgrade --force-reinstall databricks-vectorsearch
dbutils.library.restartPython()


Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
from databricks_genai_inference import ChatCompletion


response = ChatCompletion.create(model="databricks-meta-llama-3-70b-instruct",
                                messages=[{"role": "system", "content": "You are a helpful assistant."},
                                          {"role": "user","content": "Why is Databricks cool?"}],
                                max_tokens=156)
print(f"response.message:{response.message}")


response.message:Databricks is indeed a cool platform, and I'm happy to share why!

Databricks is a fast-growing, popular, and widely-used platform for working with big data and data engineering. Here are some reasons why Databricks is considered cool:

1. **Unified Analytics Platform**: Databricks provides a unified platform for data engineering, data science, and data analytics. It integrates Apache Spark, Delta Lake, and other open-source technologies to enable fast, scalable, and secure data processing.

2. **Apache Spark Native**: Databricks is built on top of Apache Spark, which is an open-source, in-memory computing engine. This allows Databricks to leverage Spark's speed, scalability, and flexibility for big data processing.

3. **Delta Lake**:


In [0]:
%sql
CREATE
OR REPLACE TABLE workspace.ragtesting.offerings2 (
  id BIGINT Generated ALWAYS AS IDENTITY,
  offer STRING
)

In [0]:
%sql
ALTER TABLE
  `workspace`.`ragtesting`.`offerings2`
SET
  TBLPROPERTIES (delta.enableChangeDataFeed = true)

In [0]:
%sql 
select * from workspace.ragtesting.offerings2

id,response


In [0]:
# Save the response to the ragtesting schema into a table called offerings
offers_df = spark.createDataFrame([
    {"offer": "I want to offer a used children's bike to collect at Berkeley. It is 4 years old and the price is $70."},
    {"offer": "I would like to sell my golfing equipment. A full set of clubs including a bag for just $300. Quality is almost as new. Collectable in the East Bay Area."},
    {"offer": "I will paint your house for just $15/hour. I'm located in San Fran and available Mon-Thur."},
    {"offer": "Used refrigerator for sale. Only $30."}
])
offers_df.write.format("delta").mode("append").option("mergeSchema", "true").saveAsTable("ragtesting.offerings2")

In [0]:
%sql
select * from workspace.ragtesting.offerings2

id,offer
1,I want to offer a used children's bike to collect at Berkeley. It is 4 years old and the price is $70.
2,I would like to sell my golfing equipment. A full set of clubs including a bag for just $300. Quality is almost as new. Collectable in the East Bay Area.
3,I will paint your house for just $15/hour. I'm located in San Fran and available Mon-Thur.
4,Used refrigerator for sale. Only $30.


In [0]:
from databricks.vector_search.client import VectorSearchClient

vsc = VectorSearchClient(disable_notice=True)

In [0]:
catalog_name = "workspace"
schema_name = "ragtesting"

In [0]:
source_table_name = "offerings2"
source_table_fullname = f"{catalog_name}.{schema_name}.{source_table_name}"

In [0]:
print(source_table_fullname)


workspace.ragtesting.offerings2


Create vector search endpoint


In [0]:
try:
    vsc.create_endpoint(
    name=vector_search_endpoint_name,
    endpoint_type="STANDARD"
    )
except Exception as e:
  if "already exists" in str(e):
    pass
  else:
    raise evector_search_endpoint_name = "vector-search-demo-endpoint"

In [0]:
# Vector index
vs_index = f"{source_table_name}_bge_index"
vs_index_fullname = f"{catalog_name}.{schema_name}.{vs_index}"

embedding_model_endpoint = "databricks-bge-large-en"

In [0]:
index = vsc.create_delta_sync_index(
  endpoint_name=vector_search_endpoint_name,
  source_table_name=source_table_fullname,
  index_name=vs_index_fullname,
  pipeline_type='TRIGGERED',
  primary_key="id",
  embedding_source_column="offer",
  embedding_model_endpoint_name=embedding_model_endpoint
)
index.describe()['status']['message']

'Delta sync Index creation is pending. Check latest status: https://dbc-f520f423-7de6.cloud.databricks.com/explore/data/workspace/ragtesting/offerings2_bge_index'

In [0]:
index.describe()

{'name': 'workspace.ragtesting.offerings2_bge_index',
 'endpoint_name': 'vector-search-demo-endpoint',
 'primary_key': 'id',
 'index_type': 'DELTA_SYNC',
 'delta_sync_index_spec': {'source_table': 'workspace.ragtesting.offerings2',
  'embedding_source_columns': [{'name': 'offer',
    'embedding_model_endpoint_name': 'databricks-bge-large-en'}],
  'pipeline_type': 'TRIGGERED',
  'pipeline_id': '0627195f-7c72-4c95-b0ad-ac0e0530a6ba'},
 'status': {'detailed_state': 'ONLINE_NO_PENDING_UPDATE',
  'message': 'Index creation succeeded. Check latest status: https://dbc-f520f423-7de6.cloud.databricks.com/explore/data/workspace/ragtesting/offerings2_bge_index',
  'indexed_row_count': 4,
  'triggered_update_status': {'last_processed_commit_version': 16,
   'last_processed_commit_timestamp': '2024-06-10T21:33:52Z'},
  'ready': True,
  'index_url': 'dbc-f520f423-7de6.cloud.databricks.com/api/2.0/vector-search/endpoints/vector-search-demo-endpoint/indexes/workspace.ragtesting.offerings2_bge_index'},

In [0]:
def retrieve_offers(query_text):
    all_columns = spark.table(source_table_fullname).columns

    results = index.similarity_search(
        query_text=query_text,
        columns=all_columns,
        num_results=2)
    
    results.update({'result': {'data_array': [result for result in results["result"]["data_array"] if result[2] > 0.55]}})

    return(results)

In [0]:
retrieve_offers(query_text="Berkeley")

{'manifest': {'column_count': 3,
  'columns': [{'name': 'id'}, {'name': 'offer'}, {'name': 'score'}]},
 'result': {'data_array': [[1.0,
    "I want to offer a used children's bike to collect at Berkeley. It is 4 years old and the price is $70.",
    0.5873685],
   [2.0,
    'I would like to sell my golfing equipment. A full set of clubs including a bag for just $300. Quality is almost as new. Collectable in the East Bay Area.',
    0.5516129]]},
 'debug_info': {'response_time': 249.0,
  'ann_time': 32.0,
  'embedding_gen_time': 107.0}}

In [0]:
results

{'manifest': {'column_count': 3,
  'columns': [{'name': 'id'}, {'name': 'offer'}, {'name': 'score'}]},
 'result': {'data_array': [[4.0,
    'Used refrigerator for sale. Only $30.',
    0.52767265]]},
 'debug_info': {'response_time': 247.0,
  'ann_time': 35.0,
  'embedding_gen_time': 76.0}}